In [24]:
import pandas as pd
import torch
from torch import nn
from torch import optim
from torch.utils.data import TensorDataset, DataLoader, random_split

### `Dataset`과 `Dataloader`

`Dataset` 클래스 기본형

In [2]:
class Dataset:
    def __init__(self, data, *arg, **kwarg):
        self.data = data
    
    def __getitem__(self, index):
        return tuple(data[index] for data in self.data.tensors)

    def __len__(self):
        return self.data[0].size(0)

#### 다중 선형 회귀

예제 3.29 - 기본 구조 선언

In [3]:
train_x = torch.FloatTensor([
    [1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7]
])
train_y = torch.FloatTensor([
    [0.1, 1.5], [1, 2.8], [1.9, 4.1], [2.8, 5.4], [3.7, 6.7], [4.6, 8]
])

예제 3.30 - 데이터세트와 데이터로더

In [4]:
train_dataset = TensorDataset(train_x, train_y)
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True, drop_last=True)

예제 3.31 - 모델, 오차 함수, 최적화 함수 선언

In [5]:
model = nn.Linear(2, 2, bias=True)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

예제 3.32 - 데이터로더 적용

In [6]:
for epoch in range(20000):
    cost = 0.0

    for batch in train_dataloader:
        x, y = batch
        output = model(x)

        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost /= len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

Epoch : 1000, Model : [Parameter containing:
tensor([[0.5529, 0.1432],
        [0.6248, 0.5299]], requires_grad=True), Parameter containing:
tensor([-0.0849,  0.2816], requires_grad=True)], Cost : 0.109
Epoch : 2000, Model : [Parameter containing:
tensor([[0.7600, 0.0363],
        [0.7724, 0.4537]], requires_grad=True), Parameter containing:
tensor([-0.3989,  0.0579], requires_grad=True)], Cost : 0.028
Epoch : 3000, Model : [Parameter containing:
tensor([[ 0.8654, -0.0183],
        [ 0.8475,  0.4148]], requires_grad=True), Parameter containing:
tensor([-0.5589, -0.0561], requires_grad=True)], Cost : 0.007
Epoch : 4000, Model : [Parameter containing:
tensor([[ 0.9191, -0.0461],
        [ 0.8858,  0.3950]], requires_grad=True), Parameter containing:
tensor([-0.6404, -0.1142], requires_grad=True)], Cost : 0.002
Epoch : 5000, Model : [Parameter containing:
tensor([[ 0.9465, -0.0602],
        [ 0.9053,  0.3849]], requires_grad=True), Parameter containing:
tensor([-0.6819, -0.1437], requires

예제 3.34 - 편향 제거

In [7]:
model = nn.Linear(2, 2, bias=False)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(20000):
    cost = 0.0

    for batch in train_dataloader:
        x, y = batch
        output = model(x)

        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost /= len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

Epoch : 1000, Model : [Parameter containing:
tensor([[ 0.7698, -0.0476],
        [ 0.6970,  0.5259]], requires_grad=True)], Cost : 0.066
Epoch : 2000, Model : [Parameter containing:
tensor([[ 0.9606, -0.2022],
        [ 0.7797,  0.4590]], requires_grad=True)], Cost : 0.041
Epoch : 3000, Model : [Parameter containing:
tensor([[ 1.1124, -0.3248],
        [ 0.8454,  0.4059]], requires_grad=True)], Cost : 0.026
Epoch : 4000, Model : [Parameter containing:
tensor([[ 1.2330, -0.4224],
        [ 0.8977,  0.3636]], requires_grad=True)], Cost : 0.017
Epoch : 5000, Model : [Parameter containing:
tensor([[ 1.3289, -0.4998],
        [ 0.9392,  0.3300]], requires_grad=True)], Cost : 0.011
Epoch : 6000, Model : [Parameter containing:
tensor([[ 1.4051, -0.5615],
        [ 0.9722,  0.3033]], requires_grad=True)], Cost : 0.007
Epoch : 7000, Model : [Parameter containing:
tensor([[ 1.4656, -0.6104],
        [ 0.9985,  0.2821]], requires_grad=True)], Cost : 0.004
Epoch : 8000, Model : [Parameter containi

### 모델/데이터세트 분리

#### 비선형회귀

예제 3.36 - 사용자 정의 데이터세트

In [9]:
class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x = df.iloc[:, 0].values
        self.y = df.iloc[:, 1].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x[index]**2, self.x[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y

    def __len__(self):
        return self.length

예제 3.37 - 사용자 정의 모델

In [10]:
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2,1)

    def forward(self, x):
        x = self.layer(x)
        return x

예제 3.38 - 사용자 정의 데이터세트와 데이터로더

In [11]:
train_dataset = CustomDataset("../data/non_linear.csv")
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=True)

예제 3.39 - GPU 연산 적용

In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)
device

'cuda'

예제 3.40 - 학습 진행

In [15]:
for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)

        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost /= len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

Epoch : 1000, Model : [Parameter containing:
tensor([[ 3.0982, -1.7012]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.5934], device='cuda:0', requires_grad=True)], Cost : 0.079
Epoch : 2000, Model : [Parameter containing:
tensor([[ 3.0985, -1.7033]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.5849], device='cuda:0', requires_grad=True)], Cost : 0.078
Epoch : 3000, Model : [Parameter containing:
tensor([[ 3.0986, -1.7034]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.5773], device='cuda:0', requires_grad=True)], Cost : 0.077
Epoch : 4000, Model : [Parameter containing:
tensor([[ 3.0986, -1.7033]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.5702], device='cuda:0', requires_grad=True)], Cost : 0.079
Epoch : 5000, Model : [Parameter containing:
tensor([[ 3.0986, -1.7031]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.5637], device='cuda:0', requires_grad=True)]

#### 모델 평가

예제 3.41 - 모델 평가

In [17]:
with torch.no_grad():
    model.eval()
    inputs = torch.FloatTensor(
        [
            [1 ** 2, 1],
            [5 ** 2, 5],
            [11 ** 2, 11]
        ]
    ).to(device)
    outputs = model(inputs)
outputs

tensor([[  1.9347],
        [ 69.5021],
        [356.8043]], device='cuda:0')

예제 3.42 - 모델 저장

In [23]:
import os

path = "../models/"
if not os.path.exists(path):
    os.makedirs(path)
torch.save(model, path + "model.pt")
torch.save(model.state_dict(), path + "model_state_dict.pt")

#### 데이터세트 분리

예제 3.44 - 데이터세트 분리

In [26]:
dataset = CustomDataset("../data/non_linear.csv")
dataset_size = len(dataset)
train_size = int(dataset_size * 0.8)
validation_size = int(dataset_size * 0.1)
test_size = dataset_size - train_size - validation_size

train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size])
print(f"Training Data Size: {len(train_dataset)}")
print(f"Validation Data Size: {len(validation_dataset)}")
print(f"Test Data Size: {len(test_dataset)}")

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=4, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True, drop_last=True)

model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)

        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost /= len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

with torch.no_grad():
    model.eval()
    for x, y in validation_dataloader:
        x = x.to(device)
        y = y.to(device)

        outputs = model(x)
        print(f"X : {x}")
        print(f"Y : {y}")
        print(f"Outputs : {outputs}")
        print("--------------------")

Training Data Size: 160
Validation Data Size: 20
Test Data Size: 20
Epoch : 1000, Model : [Parameter containing:
tensor([[ 3.1055, -1.7042]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.1135], device='cuda:0', requires_grad=True)], Cost : 0.141
Epoch : 2000, Model : [Parameter containing:
tensor([[ 3.1024, -1.7049]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.3327], device='cuda:0', requires_grad=True)], Cost : 0.094
Epoch : 3000, Model : [Parameter containing:
tensor([[ 3.1015, -1.7050]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.4259], device='cuda:0', requires_grad=True)], Cost : 0.080
Epoch : 4000, Model : [Parameter containing:
tensor([[ 3.1001, -1.7052]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.4657], device='cuda:0', requires_grad=True)], Cost : 0.077
Epoch : 5000, Model : [Parameter containing:
tensor([[ 3.0999, -1.7053]], device='cuda:0', requires_grad=True), Paramete

#### 모델 전체 저장/불러오기

예제 3.47 - 모델 불러오기

In [27]:
model = torch.load(path + "model.pt", map_location=device)
print(model)

with torch.no_grad():
    model.eval()
    inputs = torch.FloatTensor(
        [
            [1 ** 2, 1],
            [5 ** 2, 5],
            [11 ** 2, 11]
        ]
    ).to(device)
    outputs = model(inputs)
    print(outputs)

CustomModel(
  (layer): Linear(in_features=2, out_features=1, bias=True)
)
tensor([[  1.9347],
        [ 69.5021],
        [356.8043]], device='cuda:0')


C:\Users\HyeAnn\AppData\Local\Temp\ipykernel_28096\1567141737.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(path + "model.pt", map_location=device)


#### 모델 상태 저장/불러오기

예제 3.50 - 모델 상태 불러오기

In [29]:
model = CustomModel().to(device)
model_state_dict = torch.load(path+"model_state_dict.pt", map_location=device)
model.load_state_dict(model_state_dict)

with torch.no_grad():
    model.eval()
    inputs = torch.FloatTensor(
        [
            [1 ** 2, 1],
            [5 ** 2, 5],
            [11 ** 2, 11]
        ]
    ).to(device)
    outputs = model(inputs)

outputs

C:\Users\HyeAnn\AppData\Local\Temp\ipykernel_28096\2821190327.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load(path+"model_state_dict.pt", m

tensor([[  1.9347],
        [ 69.5021],
        [356.8043]], device='cuda:0')

#### 체크포인트 저장/불러오기

In [32]:
train_dataset = CustomDataset("../data/non_linear.csv")
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

checkpoint = 1

for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)

        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost /= len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")
        torch.save({
            "model" : "CustomModel",
            "epoch" : epoch,
            "model_state_dict" : model.state_dict(),
            "optimizer_state_dict" : optimizer.state_dict(),
            "cost" : cost,
            "description" : f"CustomModel checkpoint - {checkpoint}",
        }, path+f"checkpoint-{checkpoint}.pt",
        )
        checkpoint += 1

Epoch : 1000, Model : [Parameter containing:
tensor([[ 3.1086, -1.7001]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.0021], device='cuda:0', requires_grad=True)], Cost : 0.163
Epoch : 2000, Model : [Parameter containing:
tensor([[ 3.1074, -1.7026]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.0438], device='cuda:0', requires_grad=True)], Cost : 0.154
Epoch : 3000, Model : [Parameter containing:
tensor([[ 3.1069, -1.7028]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.0821], device='cuda:0', requires_grad=True)], Cost : 0.162
Epoch : 4000, Model : [Parameter containing:
tensor([[ 3.1061, -1.7028]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.1170], device='cuda:0', requires_grad=True)], Cost : 0.159
Epoch : 5000, Model : [Parameter containing:
tensor([[ 3.1054, -1.7030]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.1491], device='cuda:0', requires_grad=True)]

예제 3.52 - 체크포인트 불러오기

In [33]:
checkpoint = torch.load(path+"checkpoint-6.pt")
model.load_state_dict(checkpoint["model_state_dict"])
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
checkpoint_epoch = checkpoint["epoch"]
checkpoint_description = checkpoint["description"]
print(checkpoint_description)

for epoch in range(checkpoint_epoch + 1, 10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)

        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost /= len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

C:\Users\HyeAnn\AppData\Local\Temp\ipykernel_28096\255518434.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path+"checkpoint-6.pt")


CustomModel checkpoint - 6
Epoch : 7000, Model : [Parameter containing:
tensor([[ 3.1047, -1.7031]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.2053], device='cuda:0', requires_grad=True)], Cost : 0.109
Epoch : 8000, Model : [Parameter containing:
tensor([[ 3.1044, -1.7029]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.2300], device='cuda:0', requires_grad=True)], Cost : 0.106
Epoch : 9000, Model : [Parameter containing:
tensor([[ 3.1039, -1.7032]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.2525], device='cuda:0', requires_grad=True)], Cost : 0.104
Epoch : 10000, Model : [Parameter containing:
tensor([[ 3.1034, -1.7029]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.2731], device='cuda:0', requires_grad=True)], Cost : 0.100


✔ 오늘 새로 알게된 부분이 있다면 간략히 설명해주세요.

- `Dataset`과 `Dataloader`를 사용하는 이유 : 데이터를 학습에 직접 사용하면 모듈화, 재사용성, 가독성 등을 떨어뜨리게 된다
- `Dataset` 클래스는 `init`, `getitem`, `len` 등의 내장 method가 필요하다.
- `Dataloader`는 `batch_size`, `shuffle`, `num_workers` 등을 제공한다.
- `torch.util.data.DataLoader`의 `drop_last` : 배치 크기에 맞지 않는 배치를 제거
- 모델 구현은 `nn.Module` 클래스를 상속받아 사용한다. `init`과 `forward`만 구현하고 `backward`는 구현하지 않아도 된다. `init`에서 `super` 함수로 부모 클래스를 초기화했을 것이므로, 파이토치의 자동 미분 기능인 Autograd에서 모델의 매개변수를 역으로 전파해 자동으로 기울기 또는 변화도를 계산해준다.
- 학습 중간에 `eval` 메서드를 사용하여 모델 평가 모드를 진행한 다음 다시 학습을 진행하려는 경우 `train` 메서드를 통해 학습 모드로 변경해야 한다.
- 모델을 저장하려면 `Pickle`을 활용해 python object 구조를 binary protocol로 직렬화한다. 모델을 불러오려면 저장된 object 파일을 역직렬화해 현재 프로세스의 메모리에 업로드한다.
- 모델 전체를 저장하면 모델의 모든 정보를 저장하므로 모델 상태만 저장하는 것보다 더 많은 저장 공간이 필요하다. 모델 상태 (`torch.state_dict`)는 모델에서 학습이 가능한 매개변수를 `OrderedDict` 형식으로 반환한다.